# Adding Safeguards to our Data Extraction Workflow

In [18]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [19]:
# Load API Credentials
with open('/Users/default.DESKTOP-0LDO0LD/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [20]:
# set our API call parameters. When we want to search somthing else just change these parameters. 
LOCATION = 'NY,NY'
TERM = 'Pizza'

## Create a results-in-progress JSON file, but only if it doesn't exist.

This is the file where your results will be saved. Note: rename your JSON_FILE for different queries to prevent confusing results from other searches.

We recommend you include your search terms in the filename.

In [21]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_NY_pizza.json"

## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)

## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")


[i] Data/results_in_progress_NY_pizza.json not found. Saving empty list to file.


## Determine how many results are already in the file

In [22]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


## Figure out how many pages of results we will need

In [23]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION, term=TERM, offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [24]:
## How many results total?
total_results = results['total']
total_results

12300

In [25]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [26]:
# Import additional packages for controlling our loop
import time, math

# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

615

## Add this page of results to .json file



In [27]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses']) 
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

## Set up a progress bar in our for loop

In [29]:
import time
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.002) 

  0%|          | 0/615 [00:00<?, ?it/s]